In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Purpose: To test the neuron decomposition pipeline using 
the vdi for microns that accesses the data through the caveclient
"""

'\nPurpose: To test the neuron decomposition pipeline using \nthe vdi for microns that accesses the data through the caveclient\n'

# Setting up the Cave Client

In [3]:
from caveclient import CAVEclient
client = CAVEclient()
client.auth.get_new_token()

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.daf-apis.com/auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


In [4]:
CAVE_TOKEN  = "INSERT TOKEN HERE"

#this token is an example token but is not valid
CAVE_TOKEN="ba0a1b546e9b87f402c5d8c1d1d45bca" 

In [5]:
# Saving the token to caveclient package and as a local env file
try:
    client.auth.save_token(token=CAVE_TOKEN)
except Exception as e:
    print(e)

filepath ="./.env" 
with open(filepath,"w+") as f:
    f.write(f"CAVE_TOKEN={CAVE_TOKEN}")

Key "token" already exists in token file "/root/.cloudvolume/secrets/global.daf-apis.com-cave-secret.json"


# Importing the VDI with microns_cave interface


In [6]:
from neurd.cave_interface import CaveInterface
from pathlib import Path

client = CaveInterface(
    "minnie65_public",
    env_filepath=Path(filepath).absolute()
)
client

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [29]:
from neurd.documentation_utils import tag

@tag(['lots','of','tags'])
def example_func():
    pass

example_func.tags

['lots', 'of', 'tags']

In [27]:
"""
Purpose: Want to label certain methods
with tags so we can go and get those methods
with certain tags to then export all
or part of their docstring


label attributes as 

"""

a = 10


In [37]:
s = """
this is my summary

this is not

"""
from datasci_tools import docstring_utils as docu
docu.summary_from_docstring(s)


'this is my summary'

In [17]:
from functools import wraps
def tag(func,tag="emphasis"):
    if not hasattr(func,"tags"):
        func.tags = []
    if tag not in func.tags:
        func.tags.append(tag)
        
    return func

@document_tag
def example_func(x = 10):
    return x*2

dir(example_func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'tag']

# Accessing the data with the Cave Interface (we are just checking to make sure cave client access is working, lines of code not necessary for VDI to function)

In [ ]:
nucleus_seg_ids = client.segment_ids_with_nucleus()
nucleus_seg_ids[:10]

In [ ]:
# finding a segement id from the nuclues id
nucleus_id = 197805

nuc_df = client.neuron_nucleus_df()
seg_id = nuc_df.query(f"id_ref=={nucleus_id}")[
    "pt_root_id"].to_list()[0]
seg_id

In [ ]:
# downloading the mesh
mesh = client.mesh_from_seg_id(
    seg_id
)

In [ ]:
# downloading the synapses
syn_df = client.pre_post_df_from_seg_id(seg_id)
syn_df

# Using the VDI Microns Cave

In [10]:
# assumes your caveclient token is stored in an env file
from neurd.vdi_microns_cave import volume_data_interface as vdi

In [12]:
segment_id = 864691135502491741
syn_df = vdi.segment_id_to_synapse_df(segment_id)
syn_df

,segment_id,segment_id_secondary,synapse_id,prepost,synapse_x,synapse_y,synapse_z,synapse_size
0,864691135502491741,864691135446598290,60791196,presyn,119972,204962,25221,8540
1,864691135502491741,864691135542422980,55276068,presyn,114538,208976,25325,1480
2,864691135502491741,864691135498532371,77044514,presyn,131210,147454,26145,1884
3,864691135502491741,864691133673798118,99612983,presyn,140944,192064,25870,816
4,864691135502491741,864691136176212230,92346441,presyn,135980,207692,26255,25688
...,...,...,...,...,...,...,...,...
2258,864691135502491741,864691135575386654,82723156,postsyn,134149,174026,24229,3996
2259,864691135502491741,864691136105933529,89540754,postsyn,137552,184610,25572,9260
2260,864691135502491741,864691136617619675,104897868,postsyn,145084,190310,26798,11104
2261,864691135502491741,864691135939963393,89272330,postsyn,135678,188953,25192,900


In [13]:
syn_dict = vdi.segment_id_to_synapse_dict(segment_id)

# of presyn: 134
# of postsyn: 2129


In [14]:
syn_dict['presyn']["synapse_coordinates"]

array([[ 479888.,  819848., 1008840.],
       [ 458152.,  835904., 1013000.],
       [ 524840.,  589816., 1045800.],
       [ 563776.,  768256., 1034800.],
       [ 543920.,  830768., 1050200.],
       [ 479152.,  828928., 1004080.],
       [ 486128.,  821680., 1004440.],
       [ 599296.,  777704., 1055200.],
       [ 477672.,  833464., 1009120.],
       [ 576040.,  802680., 1069400.],
       [ 476168.,  837464., 1018520.],
       [ 570208.,  748808., 1105080.],
       [ 467956.,  836528., 1010320.],
       [ 507256.,  820864., 1014960.],
       [ 532280.,  662856., 1041720.],
       [ 455800.,  810800., 1010160.],
       [ 533184.,  818424., 1024640.],
       [ 573304.,  803384., 1067960.],
       [ 473344.,  839232., 1021160.],
       [ 519368.,  564312., 1046120.],
       [ 542720.,  830832., 1049400.],
       [ 541912.,  816064., 1028360.],
       [ 450376.,  811560., 1010000.],
       [ 447264.,  808200., 1010000.],
       [ 447224.,  812280., 1010280.],
       [ 452848.,  811960

In [ ]:
vdi.client.segment_ids_with_nucleus()

In [ ]:
vdi.voxel_to_nm_scaling

# Neuron Preprocessing Pipeline

In [ ]:
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz

In [ ]:
segment_id = 864691135502491741

# Step 0: Loading Mesh

In [ ]:
mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    plot = False
)
mesh

In [ ]:
from datasci_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 0.1 (optional): Visualizing Synapses with Mesh

In [ ]:
syn_dict = vdi.segment_id_to_synapse_dict(segment_id)

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    mesh,
    scatters = [
        syn_dict["postsyn"]["synapse_coordinates"],
        syn_dict["presyn"]["synapse_coordinates"]
    ],
    scatters_colors=["blue","yellow"]
)

# Step 1: Decimation

In [ ]:
decimation_parameters = dict(
    decimation_ratio =0.25,
)

In [ ]:
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

In [ ]:
ipvu.plot_objects(mesh_decimated)

In [ ]:
# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ),
    
)

print(products)

# Step 2: Soma Identification

In [ ]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

In [ ]:
# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

In [ ]:
# overwritting decimated mesh because original was already decimated
mesh_decimated = mesh

In [ ]:
from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

In [ ]:
sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

In [ ]:
soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

In [ ]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Step 2b: Saving off pipeline products

In [ ]:
from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

# Step 3: Decomposition

In [ ]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

In [ ]:
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

In [ ]:
decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

In [ ]:
print(neuron_obj.pipeline_products)

# Saving off neuron

In [ ]:
vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

In [ ]:
neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

In [ ]:
print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

In [ ]:
multi_soma_split_parameters = dict()

In [ ]:
_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


In [ ]:
neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

In [ ]:
neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

In [ ]:
n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

In [ ]:
from neurd import neuron_pipeline_utils as npu

In [ ]:
neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

In [ ]:
nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

In [ ]:
neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

In [ ]:
_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

In [ ]:
vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

In [ ]:
from neurd import synapse_utils as syu

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

In [ ]:
syu.plot_synapses(neuron_obj_proof)

In [ ]:
syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

In [ ]:
syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

In [ ]:
nviz.plot_compartments(neuron_obj_proof)

In [ ]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

# overwritting decimated mesh because original was already decimated
mesh_decimated = mesh

from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Step 2b: Saving off pipeline products

from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

# Step 3: Decomposition

from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

print(neuron_obj.pipeline_products)

# Saving off neuron

vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

multi_soma_split_parameters = dict()

_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

from neurd import neuron_pipeline_utils as npu

neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

from neurd import synapse_utils as syu

from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

syu.plot_synapses(neuron_obj_proof)

syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

nviz.plot_compartments(neuron_obj_proof)